In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [5]:
db = pd.read_excel("Retail-Supply-Chain-Sales-Dataset.xlsx")

In [6]:
db.sample(4)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Retail Sales People,Product ID,Category,Sub-Category,Product Name,Returned,Sales,Quantity,Discount,Profit
1329,1330,CA-2016-142762,2016-05-23,2016-05-27,Standard Class,LD-17005,Lisa DeCherney,Consumer,United States,San Francisco,...,Anna Andreadi,FUR-FU-10003691,Furniture,Furnishings,"Eldon Image Series Desk Accessories, Ebony",Not,37.050,3,0.0,16.3020
8780,8781,CA-2015-133585,2015-01-03,2015-04-03,First Class,CM-12715,Craig Molinari,Corporate,United States,Houston,...,Kelly Williams,OFF-AR-10003696,Office Supplies,Art,Panasonic KP-350BK Electric Pencil Sharpener w...,Not,55.328,2,0.2,6.2244
7577,7578,CA-2014-134726,2014-02-11,2014-07-11,Second Class,SW-20755,Steven Ward,Corporate,United States,Seattle,...,Anna Andreadi,TEC-PH-10001750,Technology,Phones,Samsung Rugby III,Yes,52.792,1,0.2,4.6193
4367,4368,CA-2017-117044,2017-11-09,2017-11-09,Second Class,HA-14920,Helen Andreada,Consumer,United States,Chicago,...,Kelly Williams,OFF-FA-10000936,Office Supplies,Fasteners,Acco Hot Clips Clips to Go,Not,10.528,4,0.2,3.2900


In [7]:
target = db.iloc[:,[4,6,7,9] + list(range(16,22))].copy()

In [8]:
target['Duration'] =db['Ship Date']-db['Order Date']

In [9]:
target['Returned']=db['Returned'].map({'Yes':1,'Not':0})
def extract_brand(product_name):
    if isinstance(product_name,str):
        return product_name.split()[0].strip(",").title()
    return "Unknown"
target['Brand']=target['Product Name'].apply(extract_brand)
target['Duration']=target['Duration'].dt.days
target.sample(5)

,Ship Mode,Customer Name,Segment,City,Sub-Category,Product Name,Returned,Sales,Quantity,Discount,Duration,Brand
1398,First Class,Nat Carroll,Consumer,New York City,Accessories,SanDisk Ultra 32 GB MicroSDHC Class 10 Memory ...,0,88.400,4,0.0,2,Sandisk
3370,Same Day,Toby Braunhardt,Consumer,Norman,Accessories,Logitech P710e Mobile Speakerphone,0,1287.450,5,0.0,0,Logitech
3935,Standard Class,Anthony Witt,Consumer,Bakersfield,Phones,Ooma Telo VoIP Home Phone System,0,201.584,2,0.2,181,Ooma
8210,Standard Class,Dario Medina,Corporate,Nashville,Appliances,Eureka The Boss Cordless Rechargeable Stick Vac,0,81.568,2,0.2,153,Eureka
7648,First Class,Fred Chung,Corporate,San Francisco,Chairs,Global Comet Stacking Arm Chair,0,1352.032,4,0.2,1,Global


In [10]:
encoding_cols = ['Ship Mode','Segment','City','Sub-Category','Brand']
for cols in encoding_cols:
    le=LabelEncoder()
    le.fit_transform(target[cols])
    target[cols]=le.fit_transform(target[cols])
target.drop(columns=['Product Name','Customer Name'],inplace= True)
target.sample(4)

,Ship Mode,Segment,City,Sub-Category,Returned,Sales,Quantity,Discount,Duration,Brand
8846,3,0,94,12,0,91.360,5,0.2,5,461
4108,1,0,329,16,0,27.414,3,0.4,0,41
5236,3,0,93,9,1,164.220,3,0.0,4,88
5233,3,1,477,5,0,605.340,6,0.0,4,300


In [12]:
X = target.drop(columns=['Returned'])
y = target['Returned']
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state = 42, stratify=y)
XGB = XGBClassifier( eval_metric = 'logloss', random_state = 42)
XGB.fit(x_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, ...)

In [13]:
y_pred = XGB.predict(x_test)
print("Accuracy: ",accuracy_score(y_test,y_pred))

Accuracy:  0.919959979989995
